# Recidivision Prediction: Machine Learning Optimization

## Importing Libraries and Dataset

In [1]:
# Import Libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf

In [2]:
# Reading the clean data csv file
df = pd.read_csv('Resources/Recidivism_Cleaned.csv', index_col=0)
df.head()

,Race_Asian,Race_Black,Race_Hispanic,Race_Native,Race_White,Age_25-34,Age_35-44,Age_45-54,Age_55 and Older,Age_Under 25,...,Offense Type_Property,Offense Type_Public Order,Offense Type_Violent,Release Type_Discharged End of Sentence,Release Type_Other,Release Type_Parole,Release Type_Paroled to Detainer,Release Type_Special Sentence,Year Released,Recidivism
0,0,0,0,0,1,0,0,0,0,1,...,0,0,1,0,0,1,0,0,2010,1
1,0,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,1,0,0,2010,1
2,0,0,0,0,1,1,0,0,0,0,...,1,0,0,0,0,1,0,0,2010,1
3,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,2010,1
4,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,2010,1


In [3]:
# All Columns
df.columns

Index(['Race_Asian', 'Race_Black', 'Race_Hispanic', 'Race_Native',
       'Race_White', 'Age_25-34', 'Age_35-44', 'Age_45-54', 'Age_55 and Older',
       'Age_Under 25', 'Offense Classification_A Felony',
       'Offense Classification_Aggravated Misdemeanor',
       'Offense Classification_B Felony', 'Offense Classification_C Felony',
       'Offense Classification_D Felony',
       'Offense Classification_Felony - Enhanced',
       'Offense Classification_Felony - Enhancement to Original Penalty',
       'Offense Classification_Felony - Mandatory Minimum',
       'Offense Classification_Other Felony',
       'Offense Classification_Other Felony (Old Code)',
       'Offense Classification_Other Misdemeanor',
       'Offense Classification_Serious Misdemeanor',
       'Offense Classification_Sexual Predator Community Supervision',
       'Offense Classification_Simple Misdemeanor',
       'Offense Classification_Special Sentence 2005', 'Offense Type_Drug',
       'Offense Type_Other', 

In [4]:
# From feature importance checked in the Random Forest Classifier, it was seen that Year Released contributes a lot while
# the offence classification types generally contributed the least.

# To help optimize the model all the offence classifications will be dropped and the Year Released will be dropped because
# it could be overfitting the data and not be general enough for points outside of the years in the dataset.

df.drop(columns = [
    'Offense Classification_Aggravated Misdemeanor',
       'Offense Classification_B Felony', 'Offense Classification_C Felony',
       'Offense Classification_D Felony',
       'Offense Classification_Felony - Enhanced',
       'Offense Classification_Felony - Enhancement to Original Penalty',
       'Offense Classification_Felony - Mandatory Minimum',
       'Offense Classification_Other Felony',
       'Offense Classification_Other Felony (Old Code)',
       'Offense Classification_Other Misdemeanor',
       'Offense Classification_Serious Misdemeanor',
       'Offense Classification_Sexual Predator Community Supervision',
       'Offense Classification_Simple Misdemeanor',
       'Offense Classification_Special Sentence 2005', 'Year Released'
], inplace = True)

## Machine Learning: Neural Network

### Splitting Data and Scaling

In [5]:
# Split our preprocessed data into our features and target arrays
y = df.Recidivism.values
X = df.drop(columns=["Recidivism"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=21, stratify=y, test_size=0.2)

In [6]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 40
hidden_nodes_layer3 = 20

nn_1 = tf.keras.models.Sequential()

# First hidden layer
nn_1.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid"))

# Second hidden layer
nn_1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# # Third hidden layer
# nn_1.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn_1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                1760      
                                                                 
 dense_1 (Dense)             (None, 40)                3240      
                                                                 
 dense_2 (Dense)             (None, 1)                 41        
                                                                 
Total params: 5,041
Trainable params: 5,041
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Compile the model
nn_1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [9]:
# Train the model
fit_model = nn_1.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
650/650 [==============================] - 1s 1ms/step - loss: 0.6220 - accuracy: 0.6650
Epoch 2/100
650/650 [==============================] - 1s 1ms/step - loss: 0.6160 - accuracy: 0.6671
Epoch 3/100
650/650 [==============================] - 1s 1ms/step - loss: 0.6157 - accuracy: 0.6675
Epoch 4/100
650/650 [==============================] - 1s 999us/step - loss: 0.6148 - accuracy: 0.6682
Epoch 5/100
650/650 [==============================] - 1s 1ms/step - loss: 0.6149 - accuracy: 0.6695
Epoch 6/100
650/650 [==============================] - 1s 1ms/step - loss: 0.6152 - accuracy: 0.6685
Epoch 7/100
650/650 [==============================] - 1s 1ms/step - loss: 0.6145 - accuracy: 0.6683
Epoch 8/100
650/650 [==============================] - 1s 986us/step - loss: 0.6144 - accuracy: 0.6697
Epoch 9/100
650/650 [==============================] - 1s 980us/step - loss: 0.6140 - accuracy: 0.6671
Epoch 10/100
650/650 [==============================] - 1s 993us/step - loss: 0.6136 

650/650 [==============================] - 1s 1ms/step - loss: 0.6031 - accuracy: 0.6731
Epoch 82/100
650/650 [==============================] - 1s 988us/step - loss: 0.6030 - accuracy: 0.6744
Epoch 83/100
650/650 [==============================] - 1s 999us/step - loss: 0.6030 - accuracy: 0.6738
Epoch 84/100
650/650 [==============================] - 1s 1ms/step - loss: 0.6030 - accuracy: 0.6731
Epoch 85/100
650/650 [==============================] - 1s 996us/step - loss: 0.6029 - accuracy: 0.6737
Epoch 86/100
650/650 [==============================] - 1s 1ms/step - loss: 0.6027 - accuracy: 0.6736
Epoch 87/100
650/650 [==============================] - 1s 1ms/step - loss: 0.6025 - accuracy: 0.6735
Epoch 88/100
650/650 [==============================] - 1s 1ms/step - loss: 0.6028 - accuracy: 0.6747
Epoch 89/100
650/650 [==============================] - 1s 1ms/step - loss: 0.6027 - accuracy: 0.6736
Epoch 90/100
650/650 [==============================] - 1s 1ms/step - loss: 0.6026 - accu

In [10]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

163/163 - 0s - loss: 0.6159 - accuracy: 0.6650 - 263ms/epoch - 2ms/step
Loss: 0.6158562898635864, Accuracy: 0.6649990081787109
